In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
rpms = [100, 200, 400, 800, 1600]
scan_files  = ["Nor-scan O2", "Nor-scan N2", "Rev-sweep O2", "Rev-sweep N2"]
# Set the path to the parent directory
parent_dir = "./data/"

list_main_dir=[a for a in os.listdir("./data/")]

files = []
df=pd.DataFrame()
for list_main_dir_i in list_main_dir:    
    all_files = np.array(os.listdir(parent_dir+list_main_dir_i))    
    for all_files_i in all_files:
        directory=parent_dir+list_main_dir_i+"/"+all_files_i
        files_all = os.listdir(directory)
        for files_i in files_all:
            files.append(directory+"/"+files_i)
selected_files=[]
for files_i in files:
    for scan_files_i in scan_files:
        for rpms_i in rpms:
            if str(rpms_i) in files_i:
                if scan_files_i in files_i and "rpm LSV" in files_i:
                    if "png"  not in files_i:
                        selected_files.append(files_i)
                        #df_ = pd.read_csv(files_i, sep='\t', header=None, skiprows=50, nrows=150)  
                        #df_=df_.iloc[:,1:2]                        
                        #df_.columns=[files_i.split("/")[-1][:-5]]
                        #df=pd.concat([df, df_], axis=0)    

selected_files=list(set(selected_files))

In [2]:
companies=["TEC10E50E-LHT","J-M Hispec 4000","TEC10F50A","NE Chem CAT","TEC10E50E-TPM","Ishifuku FC-I1","Ishifuku FC-I2",
           "TEC10V10A","TEC10E50E PCA","TEC10E50E-HT"]
gases=["O2"]#, "N2"]
scans=["Nor-scan",  "Rev-sweep"]
line_numbers_in_scans = [[30,730],[40, 988]]
#line_numbers_in_scans = [[30,50],[40, 50]]
rpms = [100, 200, 400, 800, 1600]
reproducibility=["std", "VL120", "VK117", "VK119"]

for companies_i in companies:
    for gases_i in gases:
        reproducibility_no=1
        for reproducibility_i in reproducibility:
            for scans_i, line_n in zip(scans, line_numbers_in_scans):               
                df=pd.DataFrame()
                title = f"{companies_i} {gases_i} {scans_i} {reproducibility_no}"
                plot_file_name=f"{companies_i}_{gases_i}_{scans_i}_{reproducibility_no}"
                for rpms_i in rpms:
                    for files_i in selected_files:                       
                        if str(rpms_i) +"rpm" in files_i:
                            if companies_i in files_i:
                                if  gases_i in files_i:
                                    if reproducibility_i in files_i:
                                        if scans_i in files_i:   
                                            df_ = pd.read_csv(files_i, sep='\t', header=None, skiprows=line_n[0], nrows=line_n[1]) 
                                            current=np.array(df_.iloc[:,1:2])*1000/0.1256
                                            potential=df_.iloc[:,0:1]   
                                            df_["Current density / mAcm-2"]=current
                                            df_["labels"]=[rpms_i]*len(potential)
                                            df_["Potential / V vs. RHE"]=np.array(potential).astype('float64')
                                            df=pd.concat([df, df_.iloc[:,3:6]], axis=0)                 
                try:     
                    fig = px.line(df, x="Potential / V vs. RHE", color="labels", y="Current density / mAcm-2", title=title)    
                    fig.update_layout(legend=dict(yanchor="top", y=0.7, xanchor="right", x=1), font=dict(size=16), legend_font_family="Arial")
                    fig.update_xaxes(title_font_family="Arial", title_font_color="black", titlefont=dict(size=24), tickfont=dict(size=20), tickfont_color="black")
                    fig.update_yaxes(title_font_family="Arial", title_font_color="black", titlefont=dict(size=24), tickfont=dict(size=20), tickfont_color="black")
                    fig.update_layout(title_font_family="Arial", title_font_color="black", titlefont=dict(size=24))
                    fig.write_image(f"./images/{plot_file_name}.png")
                    #fig.show()
                    #print(df)
                    #print(plot_file_name, rpms_i, files_i[-50:])
                    
                except:
                    pass
            reproducibility_no +=1
#Comparison of nor and rev scans            
for companies_i in companies:
    for gases_i in gases:
        reproducibility_no=1
        for reproducibility_i in reproducibility:
            df=pd.DataFrame()
            title = f"{companies_i} {gases_i} {reproducibility_no}"
            plot_file_name=f"{companies_i}_{gases_i}_{reproducibility_no}"            
            for scans_i, line_n in zip(scans, line_numbers_in_scans):                
                for rpms_i in rpms:
                    for files_i in selected_files:                       
                        if str(rpms_i) +"rpm" in files_i:
                            if companies_i in files_i:
                                if  gases_i in files_i:
                                    if reproducibility_i in files_i:
                                        if scans_i in files_i:                                         
                                            df_ = pd.read_csv(files_i, sep='\t', header=None, skiprows=line_n[0], nrows=line_n[1]) 
                                            current=np.array(df_.iloc[:,1:2])*1000/0.1256
                                            potential=df_.iloc[:,0:1]   
                                            df_["Current density / mAcm-2"]=current
                                            df_["labels"]= [f"{r}_{s}" for r,s in zip([rpms_i]*len(potential), [scans_i]*len(potential))]
                                            df_["Potential / V vs. RHE"]=np.array(potential).astype('float64')
                                            df=pd.concat([df, df_.iloc[:,3:6]], axis=0)                 
            try:     
                fig = px.line(df, x="Potential / V vs. RHE", color="labels", y="Current density / mAcm-2", title=title)
                fig.update_xaxes(title_font_family="Arial", title_font_color="black", titlefont=dict(size=24), tickfont=dict(size=20), tickfont_color="black")
                fig.update_yaxes(title_font_family="Arial", title_font_color="black", titlefont=dict(size=24), tickfont=dict(size=20), tickfont_color="black")
                fig.update_layout(title_font_family="Arial", title_font_color="black", titlefont=dict(size=24))
                fig.write_image(f"./images/{plot_file_name}.png")
                #fig.show()
                #print(df)
                #print(plot_file_name, rpms_i, files_i[-50:])

            except:
                pass
            reproducibility_no +=1            